# 线性回归的Pytorch框架组件实现

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
"""生成数据"""
def synthetic_data(w, b, num_examples):
    # 正态分布生成X数据集
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    # 给y加上噪音
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_W = torch.tensor([2, -3.4])
true_b = 4.2

features, labels = synthetic_data(true_W, true_b, 1000)

In [17]:
"""读取数据集"""

def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [8]:
"""定义模型"""

from torch import nn

# 定义神经网络的一层，这一层是线性模型，输入是两个特征，输出是一个值
net = nn.Sequential(nn.Linear(2, 1))

在使用net之前，我们需要初始化模型参数。 如在线性回归模型中的权重和偏置。 深度学习框架通常有预定义的方法来初始化参数。 在这里，我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零。
正如我们在构造nn.Linear时指定输入和输出尺寸一样， 现在我们能直接访问参数以设定它们的初始值。 我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。

In [11]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
net

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [13]:
"""定义损失函数"""

# 计算均方误差使用的是MSELoss类,默认情况下，它返回所有样本损失的平均值
loss = nn.MSELoss()

In [15]:

trainer = torch.optim.SGD(net.parameters(), lr=0.01)

In [20]:
"""训练"""

num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        # 清空上一轮的梯度
        trainer.zero_grad()
        l.backward()
        trainer.step() # 根据梯度和学习率，修改参数
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss: {l}')

epoch 1, loss: 0.00010130215378012508
epoch 2, loss: 0.00010122056119143963
epoch 3, loss: 0.00010126904817298055


In [21]:
w = net[0].weight.data
print(f'w的估计误差" {true_W - w.reshape(true_W.shape)}')
b = net[0].bias.data
print(f'b的估计误差: {true_b - b}')

w的估计误差" tensor([3.7467e-04, 7.9632e-05])
b的估计误差: tensor([-0.0004])
